In [19]:
import pandas as pd
import re
from scipy.stats import norm
import numpy as np
import math
import os
from fredapi import Fred
### Dataframe with the following columns. cusip, underlying, strike, time, vol
df= pd.read_csv(file_path_here)

# TEST DATA SET
data = {
    'cusip': ['sw2414','12414r','125vag'],
    'underlying': [45, 50, 55],
    'strike': [60, 45, 50],
    'time': [0.5, 0.5, 0.5],
    'vol': [0.1, 0.3, 0.2],
}
df = pd.DataFrame(data)

#### this is to get the risk free rate

def _risk_free_rate():
    # Get the API key from environment variables
    api_key = os.environ.get('fredapi')

    # Initialize the Fred API client
    fred = Fred(api_key=api_key)

    # Specify the series ID you want to retrieve
    series_id = 'GS10'

    try:
        # Retrieve the data for the series
        data = fred.get_series(series_id)
    except Exception as e:
        print(f"An error occurred: {str(e)}")
        return None

    # Get the latest interest rate and convert it to a decimal
    rate = data.iloc[-1] / 100

    df['risk_free_rate'] = rate
    return df

def _black_scholes_model(S, K, T, r, sigma):
    d1 = (math.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * math.sqrt(T))
    d2 = d1 - sigma * math.sqrt(T)
    call_price = S * norm.cdf(d1) - K * math.exp(-r * T) * norm.cdf(d2)
    put_price = K * math.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)
    return round(call_price, 3), round(put_price, 3)

def calculate_option_prices(df):
    """
    args: the dataframe that has the following 4 columns of underlying, strike, time, & vol. 
    returns: The function calculates the black scholes model row by row of a dataframe and appends 3 columns
             1st column: the risk_free_rate (from the function _append_rfr)
             2nd column: call_price which we get from the model
             3rd column: put_price which we get from the model
    nested_function:
        _black_scholes_model
        _risk_free_rate
    """
    #df= _append_rfr(df)
    df = _risk_free_rate()
    column_names = df.columns
    strike_columns = [col for col in column_names if re.search(r'strike|strike price', col, re.I)]
    rfr_column = next((col for col in column_names if re.search(r'risk_free_rate|rfr', col, re.I)), None)
    underlying_column = next((col for col in column_names if re.search(r'underlying', col, re.I)), None)
    time_column = next((col for col in column_names if re.search(r'time', col, re.I)), None)
    vol_column = next((col for col in column_names if re.search(r'volatility|vol', col, re.I)), None)

    if underlying_column is None or time_column is None or rfr_column is None or vol_column is None or not strike_columns:
        raise ValueError("Missing or incorrectly named columns in the DataFrame.")

    # Corrected code for calculating option prices
    df['call_price'] = df.apply(lambda row: _black_scholes_model(row[underlying_column], row[strike_columns[0]], row[time_column], row[rfr_column], row[vol_column])[0], axis=1)
    df['put_price'] = df.apply(lambda row: _black_scholes_model(row[underlying_column], row[strike_columns[0]], row[time_column], row[rfr_column], row[vol_column])[1], axis=1)
    return df

def create_csv(df):
    """
    args: 
        dataframe above 
    outputs:
        takes the dataframe and creates a csv
    """
    save_to_csv = input("Do you want to create a CSV file? (yes/no): ").strip().lower()

    if save_to_csv == "yes":
        file_name = input("Enter the filename (without .csv extension): ").strip() + ".csv"
        df.to_csv(file_name, index=False)
        print(f"DataFrame saved to {file_name}")
    else:
        print("DataFrame not saved to CSV.")

blk_scholes_df = calculate_option_prices(df)
create_csv(blk_scholes_df)

Do you want to create a CSV file? (yes/no):  yes
Enter the filename (without .csv extension):  this_is_cool


DataFrame saved to this_is_cool.csv
